# Model used
I'll be using llama.cpp so I need a model in GGUF format,
huggingface has lots of models and you can filter with GGUF tag

the model I will be using is 
meta-llama-3-8b-instruct.Q4_K_S.gguf which is a quantized model of llama-3-7b 
from https://huggingface.co/SanctumAI/Meta-Llama-3-8B-Instruct-GGUF 

Uncomment the line below to download it, or use huggingface hub through python api

In [1]:
# mkdir models
# !curl -o ./models https://huggingface.co/SanctumAI/Meta-Llama-3-8B-Instruct-GGUF/blob/main/meta-llama-3-8b-instruct.Q4_K_M.gguf 

In [2]:
# llm = Llama.from_pretrained(
#     repo_id="SanctumAI/Meta-Llama-3-8B-Instruct-GGUF",
#     filename="*Q4_k_M.gguf",
#     verbose=True
# )

## Imports

In [1]:
from llama_cpp import Llama
import textwrap
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import torch
import gc
import transformers
import os
from dotenv import load_dotenv

load_dotenv()


/home/userzero/.local/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-22 15:15:55.168731: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 15:15:55.200458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 15:15:55.200484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been

True

# LLM using llama.cpp without RAG

In [4]:
llm = Llama(
    model_path="./models/meta-llama-3-8b-instruct.Q4_K_M.gguf",
    n_gpu_layers=-1,
    n_ctx=4096,
    flash_attn=True

)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./models/meta-llama-3-8b-instruct.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.a

In [20]:
output = llm(
    "Who was Zed main rival in Runeterra?",
      max_tokens=None, # set to None to generate up to the end of the context window
      stop=["Q:"], # Stop generating just before the model would generate a new question
      echo=False,
      temperature=0.75
) # Generate a completion, can also call create_completion
# print(output)
print(textwrap.fill(output["choices"][0]["text"], width=70))

Llama.generate: prefix-match hit

llama_print_timings:        load time =     349.62 ms
llama_print_timings:      sample time =     201.97 ms /   300 runs   (    0.67 ms per token,  1485.38 tokens per second)
llama_print_timings: prompt eval time =     540.09 ms /   803 tokens (    0.67 ms per token,  1486.80 tokens per second)
llama_print_timings:        eval time =    5263.97 ms /   299 runs   (   17.61 ms per token,    56.80 tokens per second)
llama_print_timings:       total time =    6220.67 ms /  1102 tokens


    remove unneeded formatting and links but don't change pure text
format : Title: Kayle: Kayle, born to a Targonian Aspect at the height
of the Rune Wars, honored her mother's legacy by fighting for justice
on wings of divine flame. She and her twin sister Morgana were the
protectors of Demacia for many years—until Kayle became disillusioned
with the repeated failings of mortals, and abandoned this realm
altogether. Still, legends are told of her punishing the unjust with
her fiery swords, and many hope that she will one day return... No
human is perfect. But I am not a human. Background: Early life: Kayle
and her twin sister Morgana were born on Mount Targon during the
height of the Rune Wars to Kilam and Mihira, the current Aspect of
Justice. Due to Mihira ascending while pregnant, the sisters were born
with a fraction of her celestial power. As the war raged on, Mihira's
celestial power began to overshadow her personality and affections as
she dedicated more and more of her time t

In [69]:
del llm
_ = gc.collect()